In [1]:
import torch
import torch.nn as nn
import snntorch as snn
import numpy as np
from snntorch import import_nirtorch
import nir


In [2]:
graph = nir.read("scnn_mnist.nir")
graph.nodes.keys()

dict_keys(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9', 'input', 'output'])

In [3]:
net = import_nirtorch.from_nir(graph)

replace rnn subgraph with nirgraph


In [4]:
net

GraphExecutor(
  (0): Conv2d(2, 16, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (1): Leaky()
  (10): Leaky()
  (11): Linear(in_features=256, out_features=10, bias=True)
  (12): Leaky()
  (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Leaky()
  (4): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
  (5): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): Leaky()
  (7): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=128, out_features=256, bias=True)
  (input): Identity()
  (output): Identity()
)

In [5]:
inp_data = torch.from_numpy(np.load("val_numbers.npy")).float()
inp_data.shape

torch.Size([300, 10, 2, 34, 34])

In [6]:
modules = [e.elem for e in net.get_execution_order()]

In [9]:
# init all I&F neurons
mem_dict = {}
for idx, module in enumerate(modules):
  if isinstance(module, snn.Leaky):
    mem_dict[idx] = module.init_leaky()

out = []
for t in range(inp_data.shape[0]):
  x = inp_data[t]
  for idx, module in enumerate(modules):
    if isinstance(module, snn.Leaky):
      x, mem_dict[idx] = module(x, mem_dict[idx])
    else:
      x = module(x)
  out.append(x)
out = torch.stack(out).detach()

In [8]:
np.save("snnTorch_activity.npy", out.numpy())